In [ ]:
!pip install langchain langchain-core langchain-community langchain-ollama pydantic pandas


In [ ]:
import os
import json
import pandas as pd
from pathlib import Path
from pydantic import BaseModel, Field, field_validator
from typing import Union
from langchain.document_loaders import PyPDFLoader
from langchain_ollama import ChatOllama
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
from langchain_core.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from concurrent.futures import ThreadPoolExecutor, as_completed

class PVArticleData(BaseModel):
    title: str = Field("N/A")
    last_name: str = Field("N/A")
    year: str = Field("N/A")
    doi: str = Field("N/A")
    research_focus: str = Field("N/A")
    key_findings: str = Field("N/A")
    device_type: str = Field("N/A")
    absorber_material: str = Field("N/A")
    absorber_material_term_used: str = Field("N/A")
    absorber_dopant_material: str = Field("N/A")
    absorber_dopant_material_term_used: str = Field("N/A")
    absorber_dopant_polarity: str = Field("N/A")
    absorber_dopant_polarity_term_used: str = Field("N/A")
    front_surface_morphology: str = Field("N/A")
    front_surface_morphology_term_used: str = Field("N/A")
    rear_surface_morphology: str = Field("N/A")
    rear_surface_morphology_term_used: str = Field("N/A")
    front_surface_passivation_material: str = Field("N/A")
    front_surface_passivation_material_term_used: str = Field("N/A")
    rear_surface_passivation_material: str = Field("N/A")
    rear_surface_passivation_material_term_used: str = Field("N/A")
    negative_metallization_material: str = Field("N/A")
    negative_metallization_material_term_used: str = Field("N/A")
    positive_metallization_material: str = Field("N/A")
    positive_metallization_material_term_used: str = Field("N/A")
    efficiency_percent: str = Field("N/A")
    cell_area_cm2: str = Field("N/A")
    short_circuit_current_a: str = Field("N/A")
    short_circuit_current_density_ma_cm2: str = Field("N/A")
    open_circuit_voltage_v: str = Field("N/A")
    fill_factor_percent: str = Field("N/A")

    @field_validator("*", mode="before")
    def convert_to_string(cls, v):
        return "N/A" if v is None else str(v)

def clean_response_data(response_data):
    if isinstance(response_data, list):
        response_data = response_data[0] if response_data else {}
    if isinstance(response_data, PVArticleData):
        return response_data.model_dump()
    if isinstance(response_data, str):
        print("🧪 Cleaning string response...")
        cleaned = (
            response_data.strip()
            .removeprefix("```json").removeprefix("```")
            .removesuffix("```").strip()
        )
        try:
            parsed = json.loads(cleaned)
            return parsed[0] if isinstance(parsed, list) else parsed
        except json.JSONDecodeError:
            try:
                cleaned = cleaned.replace('\n', '').replace('\t', '').replace("'", '"')
                parsed = json.loads(cleaned)
                return parsed[0] if isinstance(parsed, list) else parsed
            except Exception as e:
                print(f"❌ JSON decode error after cleaning: {e}")
                return {}
    if isinstance(response_data, dict):
        return response_data
    print("⚠️ Unexpected response format:", type(response_data))
    return {}

# Initialize model and parser
model = ChatOllama(model="gemma3:4b")
raw_parser = PydanticOutputParser(pydantic_object=PVArticleData)
parser = OutputFixingParser.from_llm(llm=model, parser=raw_parser)

# Prompt with stronger constraints
prompt = PromptTemplate.from_template("""\
You are extracting structured data from academic articles on photovoltaic cells.
Focus ONLY on the most efficient cell mentioned in each article.

Format your response as a single **valid JSON object** only — no markdown, no commentary, no extra text.

If a value is unavailable, use "N/A".

Your output must match this schema:
{format_instructions}

ARTICLE:
{text}
""")

chain = prompt | model | parser

pdf_folder = Path("PV1-Rhea")
pdf_files = list(pdf_folder.rglob("*.pdf"))

def process_pdf(pdf_file):
    print(f"\n📄 Processing {pdf_file}...")
    try:
        loader = PyPDFLoader(str(pdf_file))
        docs = loader.load()
        full_text = "\n\n".join([doc.page_content for doc in docs])

        splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
        chunks = splitter.split_text(full_text)

        for chunk in chunks:
            try:
                response = chain.invoke({
                    "text": chunk,
                    "format_instructions": parser.get_format_instructions()
                })
                cleaned_data = clean_response_data(response)
                if cleaned_data:
                    article_data = PVArticleData(**cleaned_data)
                    print(f"✅ Successfully extracted from {pdf_file}")
                    return article_data.model_dump()
            except Exception as e:
                print(f"⛔ Error in chunk: {e}")

        print(f"⚠️ No valid data extracted from any chunk of {pdf_file}")
        return None

    except Exception as e:
        print(f"❌ Failed to process {pdf_file}: {e}")
        return None

results = []
max_workers = 4

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_pdf, pdf_file) for pdf_file in pdf_files]
    for future in as_completed(futures):
        result = future.result()
        if result:
            results.append(result)

if not results:
    print("⚠️ No PDFs were successfully processed!")
    exit()

df = pd.DataFrame(results)

column_map = {
    "title": "Title", "last_name": "Last Name", "year": "Year", "doi": "Digital Object Identifier (DOI)",
    "research_focus": "Research Focus", "key_findings": "Key Findings", "device_type": "Device Type",
    "absorber_material": "Absorber Material", "absorber_material_term_used": "Absorber Material Term Used",
    "absorber_dopant_material": "Absorber Dopant Material", "absorber_dopant_material_term_used": "Absorber Dopant Material Term Used",
    "absorber_dopant_polarity": "Absorber Dopant Polarity", "absorber_dopant_polarity_term_used": "Absorber Dopant Polarity Term Used",
    "front_surface_morphology": "Front Surface Morphology", "front_surface_morphology_term_used": "Front Surface Morphology Term Used",
    "rear_surface_morphology": "Rear Surface Morphology", "rear_surface_morphology_term_used": "Rear Surface Morphology Term Used",
    "front_surface_passivation_material": "Front Surface Passivation Material", "front_surface_passivation_material_term_used": "Front Surface Passivation Material Term Used",
    "rear_surface_passivation_material": "Rear Surface Passivation Material", "rear_surface_passivation_material_term_used": "Rear Surface Passivation Material Term Used",
    "negative_metallization_material": "Negative Metallization Material", "negative_metallization_material_term_used": "Negative Metallization Material Term Used",
    "positive_metallization_material": "Positive Metallization Material", "positive_metallization_material_term_used": "Positive Metallization Material Term Used",
    "efficiency_percent": "Efficiency (%)", "cell_area_cm2": "Cell Area (cm2)",
    "short_circuit_current_a": "Short-Circuit Current (A)", "short_circuit_current_density_ma_cm2": "Short-Circuit Current Density (mA/cm2)",
    "open_circuit_voltage_v": "Open-Circuit Voltage (V)", "fill_factor_percent": "Fill Factor (%)"
}

df = df.rename(columns=column_map)
df = df[list(column_map.values())]

output_file = "pv_extraction_results_ollama2.csv"
df.to_csv(output_file, index=False)
print(f"\n Extracted the folder to this pdf ")
print(f"\n✅ Saved extracted data to {output_file} ({len(results)} records)")
print(f"📊 Processing summary: {len(results)}/{len(pdf_files)} PDFs processed successfully")


In [ ]:
import os
import json
import pandas as pd
from pathlib import Path
from queue import Queue
from threading import Thread, Lock
from pydantic import BaseModel, Field, field_validator
from langchain.document_loaders import PyPDFLoader
from langchain_ollama import ChatOllama
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
from langchain_core.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

# -------------------------------------------------------------------
# 📦 Data Schema using Pydantic
# -------------------------------------------------------------------

class PVArticleData(BaseModel):
    title: str = Field("N/A")
    last_name: str = Field("N/A")
    year: str = Field("N/A")
    doi: str = Field("N/A")
    research_focus: str = Field("N/A")
    key_findings: str = Field("N/A")
    device_type: str = Field("N/A")
    absorber_material: str = Field("N/A")
    absorber_material_term_used: str = Field("N/A")
    absorber_dopant_material: str = Field("N/A")
    absorber_dopant_material_term_used: str = Field("N/A")
    absorber_dopant_polarity: str = Field("N/A")
    absorber_dopant_polarity_term_used: str = Field("N/A")
    front_surface_morphology: str = Field("N/A")
    front_surface_morphology_term_used: str = Field("N/A")
    rear_surface_morphology: str = Field("N/A")
    rear_surface_morphology_term_used: str = Field("N/A")
    front_surface_passivation_material: str = Field("N/A")
    front_surface_passivation_material_term_used: str = Field("N/A")
    rear_surface_passivation_material: str = Field("N/A")
    rear_surface_passivation_material_term_used: str = Field("N/A")
    negative_metallization_material: str = Field("N/A")
    negative_metallization_material_term_used: str = Field("N/A")
    positive_metallization_material: str = Field("N/A")
    positive_metallization_material_term_used: str = Field("N/A")
    efficiency_percent: str = Field("N/A")
    cell_area_cm2: str = Field("N/A")
    short_circuit_current_a: str = Field("N/A")
    short_circuit_current_density_ma_cm2: str = Field("N/A")
    open_circuit_voltage_v: str = Field("N/A")
    fill_factor_percent: str = Field("N/A")

    @field_validator("*", mode="before")
    def to_str(cls, v):
        return "N/A" if v is None else str(v)

# -------------------------------------------------------------------
# 🧹 Helper: Clean LLM output
# -------------------------------------------------------------------

def clean_response_data(response_data):
    if isinstance(response_data, PVArticleData):
        return response_data.model_dump()

    if isinstance(response_data, str):
        cleaned = response_data.strip().removeprefix("```json").removeprefix("```").removesuffix("```").strip()
        try:
            return json.loads(cleaned)
        except:
            cleaned = cleaned.replace("\n", "").replace("\t", "").replace("'", '"')
            try:
                return json.loads(cleaned)
            except:
                return {}

    if isinstance(response_data, dict):
        return response_data
    if isinstance(response_data, list) and response_data:
        return response_data[0]

    return {}

# -------------------------------------------------------------------
# 🧠 LLM Setup
# -------------------------------------------------------------------

model = ChatOllama(model="gemma3:4b")
raw_parser = PydanticOutputParser(pydantic_object=PVArticleData)
parser = OutputFixingParser.from_llm(llm=model, parser=raw_parser)

prompt = PromptTemplate.from_template("""\
You are extracting structured data from academic articles on photovoltaic cells.
Focus ONLY on the most efficient cell mentioned in each article.

Format your response as a single **valid JSON object** only — no markdown, no commentary, no extra text.
If a value is unavailable, use "N/A".

Your output must match this schema:
{format_instructions}

ARTICLE:
{text}
""")

chain = prompt | model | parser

# -------------------------------------------------------------------
# 🧵 Worker Thread
# -------------------------------------------------------------------

def process_pdf_worker(queue: Queue, results: list, lock: Lock):
    while True:
        pdf_file = queue.get()
        if pdf_file is None:
            break

        try:
            print(f"📄 Processing: {pdf_file.name}")
            loader = PyPDFLoader(str(pdf_file))
            docs = loader.load()
            full_text = "\n\n".join(doc.page_content for doc in docs)

            splitter = RecursiveCharacterTextSplitter(chunk_size=6000, chunk_overlap=200)
            chunks = splitter.split_text(full_text)

            for chunk in chunks:
                try:
                    response = chain.invoke({
                        "text": chunk,
                        "format_instructions": parser.get_format_instructions()
                    })
                    parsed = clean_response_data(response)
                    if parsed:
                        article = PVArticleData(**parsed)
                        with lock:
                            results.append(article.model_dump())
                        print(f"✅ Success: {pdf_file.name}")
                        break
                except Exception as e:
                    print(f"⚠️ Chunk error: {e}")

        except Exception as e:
            print(f"❌ Failed to load {pdf_file.name}: {e}")
        finally:
            queue.task_done()

# -------------------------------------------------------------------
# 🚀 Main Execution
# -------------------------------------------------------------------

def run_extraction(pdf_dir="PV1-Rhea", output_csv="pv_extraction_results_queue.csv", num_threads=2):
    pdf_folder = Path(pdf_dir)
    pdf_files = list(pdf_folder.rglob("*.pdf"))

    task_queue = Queue()
    results = []
    lock = Lock()

    # Start worker threads
    threads = []
    for _ in range(num_threads):
        t = Thread(target=process_pdf_worker, args=(task_queue, results, lock))
        t.start()
        threads.append(t)

    # Enqueue PDFs
    for pdf_file in pdf_files:
        task_queue.put(pdf_file)

    # Wait for all tasks to finish
    task_queue.join()

    # Stop workers
    for _ in threads:
        task_queue.put(None)
    for t in threads:
        t.join()

    # Save results
    if results:
        save_results_to_csv(results, output_csv)
        print(f"\n✅ Extracted {len(results)} / {len(pdf_files)} PDFs.")
    else:
        print("⚠️ No valid data extracted.")

# -------------------------------------------------------------------
# 💾 Save to CSV
# -------------------------------------------------------------------

def save_results_to_csv(results, output_path):
    df = pd.DataFrame(results)

    column_map = {
        "title": "Title", "last_name": "Last Name", "year": "Year", "doi": "Digital Object Identifier (DOI)",
        "research_focus": "Research Focus", "key_findings": "Key Findings", "device_type": "Device Type",
        "absorber_material": "Absorber Material", "absorber_material_term_used": "Absorber Material Term Used",
        "absorber_dopant_material": "Absorber Dopant Material", "absorber_dopant_material_term_used": "Absorber Dopant Material Term Used",
        "absorber_dopant_polarity": "Absorber Dopant Polarity", "absorber_dopant_polarity_term_used": "Absorber Dopant Polarity Term Used",
        "front_surface_morphology": "Front Surface Morphology", "front_surface_morphology_term_used": "Front Surface Morphology Term Used",
        "rear_surface_morphology": "Rear Surface Morphology", "rear_surface_morphology_term_used": "Rear Surface Morphology Term Used",
        "front_surface_passivation_material": "Front Surface Passivation Material", "front_surface_passivation_material_term_used": "Front Surface Passivation Material Term Used",
        "rear_surface_passivation_material": "Rear Surface Passivation Material", "rear_surface_passivation_material_term_used": "Rear Surface Passivation Material Term Used",
        "negative_metallization_material": "Negative Metallization Material", "negative_metallization_material_term_used": "Negative Metallization Material Term Used",
        "positive_metallization_material": "Positive Metallization Material", "positive_metallization_material_term_used": "Positive Metallization Material Term Used",
        "efficiency_percent": "Efficiency (%)", "cell_area_cm2": "Cell Area (cm2)",
        "short_circuit_current_a": "Short-Circuit Current (A)", "short_circuit_current_density_ma_cm2": "Short-Circuit Current Density (mA/cm2)",
        "open_circuit_voltage_v": "Open-Circuit Voltage (V)", "fill_factor_percent": "Fill Factor (%)"
    }

    df = df.rename(columns=column_map)
    df = df[list(column_map.values())]
    df.to_csv(output_path, index=False)
    print(f"📁 Saved to {output_path}")

# -------------------------------------------------------------------
# 🏁 Run the pipeline
# -------------------------------------------------------------------

if __name__ == "__main__":
    run_extraction()
